<a href="https://colab.research.google.com/github/Jyoti1706/Data-Science/blob/master/MLSysOps-Assignment-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Sanity Check #

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Sat Sep  7 06:17:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
  1. cuDF
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 495, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 495 (delta 152), reused 124 (delta 91), pack-reused 269 (from 1)
Receiving objects: 100% (495/495), 138.65 KiB | 5.78 MiB/s, done.
Resolving deltas: 100% (254/254), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.4 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************



In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Found existing installation: cupy-cuda12x 12.2.0
Uninstalling cupy-cuda12x-12.2.0:
  Successfully uninstalled cupy-cuda12x-12.2.0
restarting Colab...


In [1]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [1]:
# run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

  Using cached pynvml-11.5.3-py3-none-any.whl.metadata (8.8 kB)
Using cached pynvml-11.5.3-py3-none-any.whl (53 kB)
Found existing installation: cffi 1.16.0
Uninstalling cffi-1.16.0:
  Successfully uninstalled cffi-1.16.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.3/446.3 kB 11.2 MB/s eta 0:00:00
Installing RAPIDS Stable 23.12
Starting the RAPIDS install on Colab.  This will take about 15 minutes.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mamba


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    archspec-0.2.3             |     pyhd8ed1ab_0          48 KB  conda-forge
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    conda-24.7.1               |  py

#[cuDF](https://github.com/rapidsai/cudf)#

cuDF is a GPU DataFrame library for loading, joining, aggregating, filtering, and otherwise manipulating data. cuDF leverages libcudf, a blazing-fast C++/CUDA dataframe library and the Apache Arrow columnar format to provide a GPU-accelerated pandas API.


In [2]:
!pip install numpy

In [3]:
import cudf
import io, requests

# download CSV file from GitHub
url="https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

# read CSV from memory
tips_df = cudf.read_csv(io.StringIO(content))
tips_df['tip_percentage'] = tips_df['tip']/tips_df['total_bill']*100

# display average tip by dining party size
print(tips_df.groupby('size').tip_percentage.mean())

size
1    21.729202
4    14.594901
2    16.571919
3    15.215685
6    15.622920
5    14.149549
Name: tip_percentage, dtype: float64


In [5]:
import cudf
import numpy as np
import pandas as pd

#1.1 Generate random data
num_rows = 100000
data = {
    'transaction_id': np.arange(num_rows),
    'customer_id': np.random.randint(1000, size=num_rows),
    'amount': np.random.rand(num_rows) * 1000,
    'transaction_date': pd.to_datetime(np.random.choice(pd.date_range('2023-01-01', '2024-12-31', freq='D'), num_rows), format='%Y-%m-%d')
}

#1.2 Create a cuDF DataFrame
df = cudf.DataFrame(data)

#1.3 Convert transaction_date to datetime format
df['transaction_date'] = cudf.to_datetime(df['transaction_date'])

#1.4 Extract month from transaction_date
df['month'] = df['transaction_date'].dt.month

#2.1 Handle missing values in amount column
df['amount'] = df['amount'].fillna(df['amount'].median())

#2.2 Create a new feature, log_amount
df['log_amount'] = np.log(df['amount'])

print(df.head())

   transaction_id  customer_id      amount transaction_date  month  log_amount
0               0          654  232.917762       2023-08-14      8    5.450685
1               1          361  370.186787       2024-10-11     10    5.914008
2               2          631  211.025528       2024-05-28      5    5.351979
3               3          436   26.978139       2023-09-13      9    3.295027
4               4          324  941.046424       2023-04-17      4    6.846992


#

## 1. Data Features:
>➢	Generated transaction data and loaded into a cuDF DataFrame.The data includes columns such as transaction_id, customer_id, amount, transaction_date, and category.

>➢ The transaction_date column is converted to datetime format using cudf.to_datetime()

>➢ month, is created to extract the month from the transaction date.


## 2. Functionality

> ➢	Handled missing values in the amount column by filling them with the median value.
> ➢	Created a new feature, log_amount, which is the logarithm of the amount column.

## 3. Code Quality and Efficiency:
1. **cuDF**: The script leverages cuDF for GPU-accelerated data processing, which can significantly improve performance compared to traditional CPU-based dataframes.

1. **Efficiency**: By using cuDF, the script can efficiently handle large datasets.